In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [2]:
tf.__version__

'2.1.0'

In [3]:
train_file = "./train.csv"
test_file = "./test.csv"
num_cols = ["ps_reg_01", "ps_reg_02", "ps_reg_03","ps_car_12", "ps_car_13", "ps_car_14", "ps_car_15"]
ignore_cols = ["id", "target", "ps_calc_01", "ps_calc_02", "ps_calc_03", "ps_calc_04", "ps_calc_05", 
               "ps_calc_06", "ps_calc_07", "ps_calc_08", "ps_calc_09", "ps_calc_10", "ps_calc_11", 
               "ps_calc_12", "ps_calc_13", "ps_calc_14","ps_calc_15_bin", "ps_calc_16_bin", 
               "ps_calc_17_bin","ps_calc_18_bin", "ps_calc_19_bin", "ps_calc_20_bin"]

In [4]:
cfg = {
    "feature_size": None,
    "field_size": None,
    "embed_size": 128,
    "deep_nn":[256,256],
    "dropout_fm": 0,
    "dropout_deep": 0.2,
    "output_bias": None,
    "epoch": 200,
    "batch":10000,
    "split": 0.2
}

In [5]:
def overview(cfg):    
    dfTrain = pd.read_csv(train_file)
    dfTest = pd.read_csv(test_file)
    df = pd.concat([dfTrain,dfTest], sort=False)

    field_size = len(df.columns) - len(ignore_cols)
    feature_dict = {}
    feature_size = 0
    for col in df.columns:
        if col in ignore_cols:
            continue
        elif col in num_cols:
            feature_dict[col] = feature_size
            feature_size += 1
        else:
            unique_val = df[col].unique()
            feature_dict[col] = dict(zip(unique_val,range(feature_size,len(unique_val) + feature_size)))
            feature_size += len(unique_val)
    
    cfg['field_size'] = field_size
    cfg['feature_size'] = feature_size
    return dfTrain, feature_dict

In [6]:
dfTrain, feature_dict = overview(cfg)

In [7]:
def preprocess(train_df, feature_dict, cfg):
    label_df = train_df[['target']]  
    neg, pos = np.bincount(label_df.values.flatten())
    cfg['output_bias'] = np.log([pos/neg])
    train_df.drop(['target','id'],axis=1,inplace=True)
    feature_idx = train_df.copy()
    feature_val = train_df.copy()
    for col in feature_idx.columns:
        if col in ignore_cols:
            feature_idx.drop(col,axis=1,inplace=True)
            feature_val.drop(col,axis=1,inplace=True)
            continue
        elif col in num_cols:
            feature_idx[col] = feature_dict[col]
        else:
            feature_idx[col] = feature_idx[col].map(feature_dict[col])
            feature_val[col] = 1      
            
    train_idx_df, test_idx_df = train_test_split(feature_idx, test_size=cfg["split"])
    train_val_df, test_val_df = train_test_split(feature_val, test_size=cfg["split"])
    train_label_df, test_label_df = train_test_split(label_df, test_size=cfg["split"])
    
    train_idx_df, validate_idx_df = train_test_split(train_idx_df, test_size=cfg["split"])
    train_val_df, validate_val_df = train_test_split(train_val_df, test_size=cfg["split"])
    train_label_df, validate_label_df = train_test_split(train_label_df, test_size=cfg["split"])
    
    train_input = [train_idx_df.values, train_val_df.values]
    train_label = np.array(train_label_df['target'])
    bool_train_labels = train_label != 0
    
    validate_input = [validate_idx_df.values, validate_val_df.values]
    validate_label = validate_label_df.values
    
    test_input = [test_idx_df.values, test_val_df.values]
    test_label = test_label_df.values
    
    return train_input, train_label, bool_train_labels, validate_input, validate_label, test_input, test_label

In [8]:
train_input, train_label, bool_train_labels, validate_input, validate_label, test_input, test_label = preprocess(dfTrain, feature_dict, cfg)

In [9]:
def oversample(train_input, train_label, bool_train_labels):
    pos_idx = train_input[0][bool_train_labels]
    neg_idx = train_input[0][~bool_train_labels]
    pos_val = train_input[1][bool_train_labels]
    neg_val = train_input[1][~bool_train_labels]
    pos_label = train_label[bool_train_labels]
    neg_label = train_label[~bool_train_labels]
    
    ids = np.arange(len(pos_idx))
    choices = np.random.choice(ids, len(neg_idx))
    
    res_pos_idx = pos_idx[choices]
    res_pos_val = pos_val[choices]
    res_pos_label = pos_label[choices]
    
    resampled_idx = np.concatenate([res_pos_idx, neg_idx], axis=0)
    resampled_val = np.concatenate([res_pos_val, neg_val], axis=0)
    resampled_label = np.concatenate([res_pos_label, neg_label], axis=0)

    order = np.arange(len(resampled_label))
    np.random.shuffle(order)    
    return [resampled_idx[order], resampled_val[order]], resampled_label[order]

In [10]:
res_train_input, res_train_label = oversample(train_input, train_label, bool_train_labels)

In [11]:
class DeepFM(tf.keras.Model):
    def __init__(self, cfg):
        super(DeepFM, self).__init__()
        self.feature_size = cfg['feature_size']
        self.field_size = cfg['field_size']
        self.embed_size = cfg['embed_size']
        self.deep_nn = cfg['deep_nn']
        
        self.dropout_fm = cfg['dropout_fm']
        self.dropout_deep = cfg['dropout_deep']
        
        # fm        
        self.feature_weight = tf.keras.layers.Embedding(cfg['feature_size'], 1)
        self.feature_embed = tf.keras.layers.Embedding(cfg['feature_size'], cfg['embed_size'])

        # dnn
        for layer in range(len(cfg['deep_nn'])):
            setattr(self, 'dense_' + str(layer), tf.keras.layers.Dense(self.deep_nn[layer]))
            setattr(self, 'batchNorm_' + str(layer), tf.keras.layers.BatchNormalization())
            setattr(self, 'activation_' + str(layer), tf.keras.layers.Activation('relu'))
            setattr(self, 'dropout_' + str(layer), tf.keras.layers.Dropout(self.dropout_deep))
            
        self.fc = tf.keras.layers.Dense(1, activation='sigmoid', 
                                        bias_initializer=tf.keras.initializers.Constant(cfg['output_bias']))

    def call(self, inputs, training=True):
        # inputs = [feature_idx, feature_val]
        reshaped_feature_val = tf.cast(tf.reshape(inputs[1], shape=[-1,self.field_size,1]), tf.float32)
        # linear        
        weights = self.feature_weight(inputs[0])
        linear = tf.reduce_sum(tf.multiply(weights,reshaped_feature_val),2)
        
        # fm  
        embeddings = self.feature_embed(inputs[0])
        second_inner = tf.multiply(embeddings,reshaped_feature_val)
        
        summed_features_emb = tf.reduce_sum(second_inner,1)
        summed_features_emb_square = tf.square(summed_features_emb)
        
        squared_features_emb = tf.square(second_inner)
        squared_sum_features_emb = tf.reduce_sum(squared_features_emb,1)
        
        fm = 0.5 * tf.subtract(summed_features_emb_square,squared_sum_features_emb)
        
        # dnn
        y_deep = tf.reshape(embeddings,shape=[-1,self.field_size * self.embed_size])
        for layer in range(0, len(self.deep_nn)):
            y_deep = getattr(self, 'dense_' + str(layer))(y_deep)
            y_deep = getattr(self, 'batchNorm_' + str(layer))(y_deep, training=training)
            y_deep = getattr(self, 'activation_' + str(layer))(y_deep)
            y_deep = getattr(self, 'dropout_' + str(layer))(y_deep, training=training)
            
        # concat
        concat = tf.concat([linear, fm, y_deep], axis=1)                                
        out = self.fc(concat)
        return out

In [12]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_auc', verbose=1, patience=10, mode='max',restore_best_weights=True)
metrics = [tf.keras.metrics.TruePositives(name='tp'),
           tf.keras.metrics.FalsePositives(name='fp'),
           tf.keras.metrics.TrueNegatives(name='tn'),
           tf.keras.metrics.FalseNegatives(name='fn'), 
           tf.keras.metrics.BinaryAccuracy(name='bin_acc'),
           tf.keras.metrics.Precision(name='precision'),
           tf.keras.metrics.Recall(name='recall'),
           tf.keras.metrics.AUC(name='auc')]

In [13]:
model = DeepFM(cfg)
model.compile(optimizer = tf.keras.optimizers.Adam(lr=1e-3), 
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
              metrics=metrics)
model.fit(res_train_input, res_train_label, epochs=cfg['epoch'], batch_size=cfg['batch'], shuffle=True, 
          verbose=1, callbacks = [early_stopping], validation_data=(validate_input, validate_label))

Train on 733910 samples, validate on 95234 samples
Epoch 1/200
733910/733910 [==============================] - 14s 19us/sample - loss: 0.6891 - tp: 16206.0000 - fp: 6646.0000 - tn: 360309.0000 - fn: 350749.0000 - bin_acc: 0.5130 - precision: 0.7092 - recall: 0.0442 - auc: 0.5451 - val_loss: 0.7140 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 91853.0000 - val_fn: 3381.0000 - val_bin_acc: 0.9645 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.4980
Epoch 2/200
733910/733910 [==============================] - 9s 12us/sample - loss: 0.6777 - tp: 64345.0000 - fp: 22111.0000 - tn: 344844.0000 - fn: 302610.0000 - bin_acc: 0.5575 - precision: 0.7443 - recall: 0.1753 - auc: 0.5898 - val_loss: 0.7590 - val_tp: 0.0000e+00 - val_fp: 31.0000 - val_tn: 91822.0000 - val_fn: 3381.0000 - val_bin_acc: 0.9642 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5034
Epoch 3/200
733910/733910 [==============================] - 8s 11us/sample - loss: 0.6674 - tp: 96372

In [14]:
# model.save_weights('base_pramas.h5', overwrite=True)

In [15]:
model.summary()

Model: "deep_fm"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  257       
_________________________________________________________________
embedding_1 (Embedding)      multiple                  32896     
_________________________________________________________________
dense (Dense)                multiple                  1212672   
_________________________________________________________________
batch_normalization (BatchNo multiple                  1024      
_________________________________________________________________
activation (Activation)      multiple                  0         
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
dense_1 (Dense)              multiple                  6579

In [16]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model') 

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [17]:
demodemo = tf.keras.models.load_model('saved_model/my_model')

In [18]:
demodemo.summary()

Model: "deep_fm"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  257       
_________________________________________________________________
embedding_1 (Embedding)      multiple                  32896     
_________________________________________________________________
dense (Dense)                multiple                  1212672   
_________________________________________________________________
batch_normalization (BatchNo multiple                  1024      
_________________________________________________________________
activation (Activation)      multiple                  0         
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
dense_1 (Dense)              multiple                  6579